## Exercise 1B: Keras and Dense Networks
#### Marohom, Alec Louie A. | 2019190226

#### Instructions:
Using Keras, train a network to solve a non-linear function Ax^2 + By^2 + Cz^2 + D. Generate 1000 random samples (0-1) using 20% of the data for a test set. Generate a check point at each epoch and use early stopping [1] to determine when the training should stop. Use the appropriate loss function for the problem. You can use the Keras and Deep Dense Networks notebooks as reference. Submit the notebooks (b, c, d) and this document containing the answers.

Note:
[1]: This means you need a validation data. It’s up to you to select the validation split that works for you.

b.	Train the network using a configuration of your choice that will result into a mean absolute error of less than 0.01 for the testing set. You don’t need to specifically choose mean absolute error as your loss as you can also just compute in separately. 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split as tts

In [2]:
a = 1 #Alec 
c = 13 #Marohom
b = c - a
d = -b

nSamples = 1000

In [3]:
def fxn(x, y, z):
    return (a*(x**2)) + (b*(y**2)) + (c*(z**2)) + d

    datalist = np.random.rand(1000, 3) # random data generated
    data1b = pd.DataFrame(datalist, columns=['X','Y','Z'])
    data1b.to_csv('ex_1B_data.csv', index=False)

In [4]:
datadf = pd.read_csv('ex_1B_data.csv')
X = datadf.values

In [5]:
X_train, X_test = tts(X, random_state=2020, test_size=0.2)

In [6]:
y_train = fxn(X_train[:,0],X_train[:,1],X_train[:,2])
y_test = fxn(X_test[:,0],X_test[:,1],X_test[:,2])

In [7]:
input_ = tf.keras.layers.Input(shape=(3,)) # input shape

dense1 = tf.keras.layers.Dense(500, activation='relu')(input_)
dense2 = tf.keras.layers.Dense(500, activation='relu')(dense1)

output = tf.keras.layers.Dense(1)(dense2)

In [8]:
model = tf.keras.Model(inputs=input_, outputs=output)

In [9]:
class EarlyStoppingAt(tf.keras.callbacks.Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        if logs.get(self.monitor) <= self.value:
            self.model.stop_training = True

In [10]:
filepath = r'models/'

cbList = [
    EarlyStoppingAt(monitor='val_loss', value=0.01),
    #tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=0.01, patience=5)
    tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',save_freq='epoch')
         ]

In [11]:
model.compile('adam', loss='mse', metrics=['mae'])

In [12]:
model.fit(X_train, y_train, batch_size=1, epochs=50, validation_data=(X_test,y_test), callbacks=cbList)

Train on 800 samples, validate on 200 samples
Epoch 1/50
777/800 [============================>.] - ETA: 0s - loss: 8.7829 - mae: 2.0905WARNING:tensorflow:From C:\Users\Alec\anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/assets
800/800 [==============================] - 2s 3ms/sample - loss: 8.5565 - mae: 2.0516 - val_loss: 0.9431 - val_mae: 0.7382
Epoch 2/50
800/800 [==============================] - 2s 3ms/sample - loss: 0.7523 - mae: 0.6459 - val_loss: 0.1641 - val_mae: 0.2742
Epoch 3/50
800/800 [==============================] - 2s 3ms/sample - loss: 0.1411 - mae: 0.2873 - val_loss: 0.1508 - val_mae: 0.3249
Epoch 4/50
800/800 [==============================] - 2s 3m

In [13]:
predictions = model.predict(X_test)

In [14]:
model.evaluate(X_test, y_test)

200/200 [==============================] - 0s 369us/sample - loss: 0.0058 - mae: 0.0566


[0.005803894866257906, 0.056577876]